## 필요한 패키지 및 기본 제공 함수 (Requirements)

In [ ]:
!pip install PyKomoran
!pip install nltk

     |████████████████████████████████| 6.4MB 9.9MB/s 
     |████████████████████████████████| 204kB 39.3MB/s 


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import json
import torch
from tqdm import trange
import numpy as np
import random
from nltk import sent_tokenize
from PyKomoran import *

In [ ]:
def load_data(path) :
  with open(path) as f :
    data = json.load(f)

  return data


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
# end of set_seed

## 데이터 로드 및 전처리 (Preprocess)

In [ ]:
train_path = "./newsdata_train.json"
test_path = "./newsdata_test.json"
label_list = ['IT', '경제', '문화', '스포츠', '정치']

train_data = load_data(train_path)
test_data = load_data(test_path)

komoran = Komoran("EXP")

In [ ]:
def create_w2i_l2i_i2l(data, label_list, komoran) :

    labels2idx = {label: i for i, label in enumerate(label_list)}
    idx2labels = {i: label for i, label in enumerate(label_list)}

    POS_result = set()

    for doc in data:
        sentences = sent_tokenize(doc["content"])
        for sentence in sentences:
            POS_result.update(komoran.get_plain_text(sentence).split(' '))

    word2idx = {'<PAD>': 0, '<UNK>': 1}
    for idx, token in enumerate(sorted(POS_result)):
        word2idx[token] = idx + 2

    return word2idx, labels2idx, idx2labels

In [ ]:
word2idx, labels2idx, idx2labels = create_w2i_l2i_i2l(train_data, label_list, komoran)

In [ ]:
def convert_examples_to_features(data, word2idx, labels2idx, komoran, max_length=512):
    input_ids = list()
    labels = list()

    for doc in data:
        doc_ids = []
        sentences = sent_tokenize(doc["content"])
        for sentence in sentences:
            doc_ids.extend(
                [
                    word2idx[w if w in word2idx else '<UNK>']
                    for w in komoran.get_plain_text(sentence).split(' ')
                ]
            )

        if len(doc_ids) < max_length:
            doc_ids += [word2idx['<PAD>']] * (max_length - len(doc_ids))
            
        elif len(doc_ids) > max_length:
            doc_ids = doc_ids[:max_length]

        input_ids.append(doc_ids)
        labels.append(labels2idx[doc["topic"]])

    return input_ids, labels


def make_dataset(input_ids, labels):
    return torch.utils.data.TensorDataset(torch.tensor(input_ids, dtype=torch.long),
                                          torch.tensor(labels, dtype=torch.long))

In [ ]:
train_inputs, train_labels = convert_examples_to_features(train_data, word2idx, labels2idx, komoran, max_length=512)
train_dataset = make_dataset(train_inputs, train_labels)

test_inputs, test_labels = convert_examples_to_features(test_data, word2idx, labels2idx, komoran, max_length=512)
test_dataset = make_dataset(test_inputs, test_labels)

## 모델 학습 (Train)

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self, vocab_size, output_dim):
        super(CNN, self).__init__()
        self.word_embed = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=128, padding_idx=0)
        self.conv_layer1 = torch.nn.Conv1d(128, 30, 3)
        self.conv_layer2 = torch.nn.Conv1d(128, 30, 4)
        self.conv_layer3 = torch.nn.Conv1d(128, 30, 5)
        # self.convs = torch.nn.ModuleList([torch.nn.Conv1d(128, 30, k) for k in [3, 4, 5]])
        self.dropout = torch.nn.Dropout(0.1)
        self.fc = torch.nn.Linear(3 * 30, output_dim, bias=True)

    def forward(self, inputs):
      
        embedded = self.word_embed(inputs).permute(0, 2, 1)

        conv1 = torch.nn.functional.relu(self.conv_layer1(embedded).permute(0, 2, 1).max(1)[0])

        

        conv2 = torch.nn.functional.relu(self.conv_layer2(embedded).permute(0, 2, 1).max(1)[0])
        conv3 = torch.nn.functional.relu(self.conv_layer3(embedded).permute(0, 2, 1).max(1)[0])

        x = torch.cat([conv1, conv2, conv3], dim=1)

        output = self.fc(self.dropout(x))

        return output

In [ ]:
def train(model, train_dataset, args):

    set_seed(42)

    train_batch_size = args["train_batch_size"]
    num_train_epochs = args["num_train_epochs"]
    device = args["device"]
    learning_rate = args["learning_rate"]

    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Loss function
    criterion = torch.nn.CrossEntropyLoss()

    train_dataLoader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

    train_iterator = trange(num_train_epochs, desc="Epoch")

    print("\n***** Running training *****")
    print("  Num examples = {}".format(len(train_dataset)))
    print("  Num Epochs = {}".format(num_train_epochs))
    print("  Train Batch size = {}".format(train_batch_size))
    print("  Device = ", device)

    model.to(device)
    model.train(True)
    model.zero_grad()
    for epoch in train_iterator:
        loss = 0
        for batch in train_dataLoader:
            input_vector = batch[0].to(device)
            label = batch[1].to(device)
            predict = model(input_vector)

            loss = criterion(predict, label)
            loss += loss.item()

            loss.backward()
            optimizer.step()
            model.zero_grad()

        if (epoch + 1) % 10 == 0:
            print("\n********** Train Result **********")
            print("  Epoch / Total Epoch : {} / {}".format(epoch + 1, num_train_epochs))
            print("  Loss : {:.4f}".format(loss))

    model.train(False)
# end of train

In [ ]:
vocab_size = len(word2idx)
output_dim = len(label_list)

args = dict()
args["train_batch_size"] = 64
args["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args["learning_rate"] = 0.0005
args["num_train_epochs"] = 500

model = CNN(vocab_size, output_dim)

train(model, train_dataset, args)

Epoch:   0%|          | 0/500 [00:00<?, ?it/s]


***** Running training *****
  Num examples = 800
  Num Epochs = 500
  Train Batch size = 64
  Device =  cuda


Epoch:   2%|▏         | 11/500 [00:14<04:08,  1.97it/s]


********** Train Result **********
  Epoch / Total Epoch : 10 / 500
  Loss : 1.8675


Epoch:   4%|▍         | 21/500 [00:15<01:18,  6.07it/s]

...


********** Train Result **********
  Epoch / Total Epoch : 480 / 500
  Loss : 0.0004


Epoch:  98%|█████████▊| 491/500 [01:11<00:01,  8.78it/s]


********** Train Result **********
  Epoch / Total Epoch : 490 / 500
  Loss : 0.0003


Epoch: 100%|██████████| 500/500 [01:12<00:00,  6.88it/s]


********** Train Result **********
  Epoch / Total Epoch : 500 / 500
  Loss : 0.0002


## 모델 학습 후 평가 (Evaluation)

In [ ]:
def evaluate(model, test_dataset, args, news_num=900):
    test_dataLoader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=1)

    device = args["device"]

    print("***** Running evaluation *****")
    print("  Num examples = {}".format(len(test_dataset)))
    print("  Test Batch size = 1")

    model.eval()
    pred = None
    label = None
    for batch in test_dataLoader:
        input_vector = batch[0].to(device)

        with torch.no_grad():
            predict = model(input_vector)

        if pred is None:
            pred = predict.detach().cpu().numpy()
            label = batch[1].numpy()
        else:
            pred = np.append(pred, predict.detach().cpu().numpy(), axis=0)
            label = np.append(label, batch[1].numpy(), axis=0)

    pred = np.argmax(pred, axis=1)

    news_num -= 800
    sample_pred = pred[news_num]
    sample_label = label[news_num]
    sample_result = {"pred": sample_pred, "label": sample_label}

    accuracy = (pred == label).sum() / 200

    return accuracy, sample_result

In [ ]:
accuracy, sample_result = evaluate(model, test_dataset, args, news_num=900)

print("\n********** Total Test Result **********")
print("  Accuracy {}".format(accuracy))
print("  Sample pred : {}".format(sample_result["pred"]))
print("  Sample label : {}".format(sample_result["label"]))

***** Running evaluation *****
  Num examples = 200
  Test Batch size = 1

********** Total Test Result **********
  Accuracy 0.75
  Sample pred : 0
  Sample label : 3


## 전체 코드

In [ ]:
# main

train_path = "./newsdata_train.json"
test_path = "./newsdata_test.json"
label_list = ['IT', '경제', '문화', '스포츠', '정치']

train_data = load_data(train_path)
test_data = load_data(test_path)

komoran = Komoran("EXP")

args = dict()
args["train_batch_size"] = 64
args["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args["learning_rate"] = 0.0005
args["num_train_epochs"] = 500

word2idx, labels2idx, idx2labels = create_w2i_l2i_i2l(train_data, label_list, komoran)

train_inputs, train_labels = convert_examples_to_features(train_data, word2idx, labels2idx, komoran, max_length=512)
train_dataset = make_dataset(train_inputs, train_labels)

test_inputs, test_labels = convert_examples_to_features(test_data, word2idx, labels2idx, komoran, max_length=512)
test_dataset = make_dataset(test_inputs, test_labels)

input_dim = len(word2idx)
output_dim = len(label_list)

model = CNN(input_dim, output_dim)

train(model, train_dataset, args)

accuracy, sample_result = evaluate(model, test_dataset, args, news_num=900)

print("\n********** Total Test Result **********")
print("  Accuracy {}".format(accuracy))
print("  Sample pred : {}".format(sample_result["pred"]))
print("  Sample label : {}".format(sample_result["label"]))